In [1]:
import pandas.io.sql
import pyarrow
import psycopg2
import json
import pandas as pd

In [2]:
# conn = psycopg2.connect(
#         host='localhost',
#         database='walmart',
#         user='postgres',
#         password='123456lol'
#     )

# cur = conn.cursor()

In [ ]:
# product_query = 'SELECT p.title AS title, p.id AS id, p.category_id AS catid FROM product p'
# cat_query = 'SELECT c.name AS name, c.id AS id, c.parent_id AS parent_id FROM category c'

# df= pandas.io.sql.read_sql(cat_query, conn)
# df.to_parquet('category_list.parquet', engine='pyarrow')

In [2]:
cat = pd.read_parquet('category_list.parquet', engine='pyarrow')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(cat)

                                                  name   id  parent_id
0                                 Arts Crafts & Sewing    1        NaN
1                                         Auto & Tires    2        NaN
2                                                 Baby    3        NaN
3                                               Beauty    4        NaN
4                                                Books    5        NaN
5                                          Cell Phones    6        NaN
6                                       Character Shop    7        NaN
7                                             Clothing    8        NaN
8                                         Collectibles    9        NaN
9                                          Electronics   10        NaN
10                                             Feature   11        NaN
11                                                Food   12        NaN
12                                              Health   13        NaN
13    

In [3]:
from collections import namedtuple

Node = namedtuple('Node', ['name', 'db_id', 'children'], defaults={'children': []})

# Build the tree. Root node has no name and  ID = -1 to completely avoid
# collision risks
root_node = Node('', -1, [])
# Example table hierarchy
# A:
#   B:
#       C
#       D
#   E:
#       F:
#           G
# Treat this table as a fake SQL view. Note how the order is jumbled.
category_sql_table = cat
category_sql_table = category_sql_table.reset_index()
# We might be stuck with a for-loop, as the body uses code with side effects.
# We must use some sort of storage as we cannot be guaranteed to be able to
# iterate through the table in a hierarchical manner.
# As such, this node dict allows for easy access to all nodes at once, no
# matter the iteration order.
node_dict = {
    -1: root_node
}
# Auxillary dict to keep track of child nodes, as we would often reach a child
# node before the parent node is discovered.
# The root node need not have an entry here as we already have it created.
children_dict = {}



In [4]:
for row in category_sql_table.itertuples():
    print(row)
    print(pd.isna(row[4]))
    print(pd.isna(2.0))
    break

Pandas(Index=0, index=0, name='Arts Crafts & Sewing', id=1, parent_id=nan)
True
False


In [5]:
for row in category_sql_table.itertuples():
    print(row)
    node = Node(row[2], row[3], [])
    node_dict[row[3]] = node
    if pd.isna(row[4]) == True:
        root_node.children.append(node)
    else:
        try:
            children_dict[row[4]].append(node)
        except KeyError:
            children_dict[row[4]] = [node]

Pandas(Index=0, index=0, name='Arts Crafts & Sewing', id=1, parent_id=nan)
Pandas(Index=1, index=1, name='Auto & Tires', id=2, parent_id=nan)
Pandas(Index=2, index=2, name='Baby', id=3, parent_id=nan)
Pandas(Index=3, index=3, name='Beauty', id=4, parent_id=nan)
Pandas(Index=4, index=4, name='Books', id=5, parent_id=nan)
Pandas(Index=5, index=5, name='Cell Phones', id=6, parent_id=nan)
Pandas(Index=6, index=6, name='Character Shop', id=7, parent_id=nan)
Pandas(Index=7, index=7, name='Clothing', id=8, parent_id=nan)
Pandas(Index=8, index=8, name='Collectibles', id=9, parent_id=nan)
Pandas(Index=9, index=9, name='Electronics', id=10, parent_id=nan)
Pandas(Index=10, index=10, name='Feature', id=11, parent_id=nan)
Pandas(Index=11, index=11, name='Food', id=12, parent_id=nan)
Pandas(Index=12, index=12, name='Health', id=13, parent_id=nan)
Pandas(Index=13, index=13, name='Home', id=14, parent_id=nan)
Pandas(Index=14, index=14, name='Home Improvement', id=15, parent_id=nan)
Pandas(Index=15, in

In [8]:
node_dict

{-1: Node(name='', db_id=-1, children=[Node(name='Arts Crafts & Sewing', db_id=1, children=[]), Node(name='Auto & Tires', db_id=2, children=[]), Node(name='Baby', db_id=3, children=[]), Node(name='Beauty', db_id=4, children=[]), Node(name='Books', db_id=5, children=[]), Node(name='Cell Phones', db_id=6, children=[]), Node(name='Character Shop', db_id=7, children=[]), Node(name='Clothing', db_id=8, children=[]), Node(name='Collectibles', db_id=9, children=[]), Node(name='Electronics', db_id=10, children=[]), Node(name='Feature', db_id=11, children=[]), Node(name='Food', db_id=12, children=[]), Node(name='Health', db_id=13, children=[]), Node(name='Home', db_id=14, children=[]), Node(name='Home Improvement', db_id=15, children=[]), Node(name='Household Essentials', db_id=16, children=[]), Node(name='Industrial & Scientific', db_id=17, children=[]), Node(name='Jewelry', db_id=18, children=[]), Node(name='Music', db_id=19, children=[]), Node(name='Musical Instruments', db_id=20, children=[

In [7]:
# Now with all nodes visited, iterate through children_dict
# to connect the dots.
for parent_id, child_list in children_dict.items():
    node_dict[parent_id] = node_dict[parent_id]._replace(children = child_list)

In [ ]:
#THIS IS WHERE WE CURRENTLY ARE
node_dict

In [116]:
# We now have a full tree, accessible through root_node.
# Example per-level traversal. This is an algorithm I thought up on the spot
# to serve our needs. If you can think of anything better, or suspect that
# I had a bad case of brainfart in that some textbook algorithm could've done
# the job please let me know. I desperately need sleep.
from queue import Queue

even_queue = Queue()
odd_queue = Queue()

even_queue.put(root_node)

depth = 0  # which queue to pop depends on depth

while not even_queue.empty() or not odd_queue.empty():
    pop_queue = even_queue if depth % 2 == 0 else odd_queue
    put_queue = odd_queue if depth % 2 == 0 else even_queue

    while not pop_queue.empty():
        node = pop_queue.get()
        # --- The main task (indexing, pushing to parent_of, whatever) --- #
        for child in node.children:
            put_queue.put(child)
    depth += 1